https://github.com/Coding-Crashkurse/LangGraph-Tutorial



In [2]:
def read_code_from_file(file_path: str) -> str:
    with open(file_path, "r") as file:
        code = file.read()
    return code


def write_code_to_file(file_path: str, code: str):
    with open(file_path, "w") as file:
        file.write(code)

In [3]:
from typing import Annotated
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import tool

repl = PythonREPL()


@tool
def python_repl(code: Annotated[str, "filename to read the code from"]):
    """Use this to execute python code read from a file. If you want to see the output of a value,
    Make sure that you read the code from correctly
    you should print it out with `print(...)`. This is visible to the user."""

    try:
        result = repl.run(code)
        print("RESULT CODE EXECUTION:", result)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Executed:\n```python\n{code}\n```\nStdout: {result}"

In [4]:
python_repl.invoke("./testfile.py")

Python REPL can execute arbitrary code. Use with caution.


RESULT CODE EXECUTION: SyntaxError('invalid syntax', ('<string>', 1, 1, './testfile.py\n', 1, 2))


"Executed:\n```python\n./testfile.py\n```\nStdout: SyntaxError('invalid syntax', ('<string>', 1, 1, './testfile.py\\n', 1, 2))"

In [5]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage

load_dotenv()

True

In [6]:
groq_api_key = os.getenv("GROQ_API_KEY")

# MODEL_NAME = "llama-3.1-70b-versatile"
MODEL_NAME = "mixtral-8x7b-32768"

llm = ChatGroq(
    temperature=0,
    model_name=MODEL_NAME,
    groq_api_key=groq_api_key
)

In [7]:
model_with_tools = llm.bind_tools([python_repl])

In [8]:
messages = [
    SystemMessage(
        content=""" Your task is to test code by executing it using the python_repl tool.
        If the code produced an error just return 'False'. If it worked, return 'True'"""
    ),
    HumanMessage(content="./testfile.py"),
]

In [9]:
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

In [10]:
messages

[SystemMessage(content=" Your task is to test code by executing it using the python_repl tool.\n        If the code produced an error just return 'False'. If it worked, return 'True'"),
 HumanMessage(content='./testfile.py'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3c29', 'function': {'arguments': '{"code":"./testfile.py"}', 'name': 'python_repl'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 1272, 'total_tokens': 1365, 'completion_time': 0.146491943, 'prompt_time': 0.114541556, 'queue_time': None, 'total_time': 0.261033499}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-30c6f509-d2e9-4699-ac18-85d70173002c-0', tool_calls=[{'name': 'python_repl', 'args': {'code': './testfile.py'}, 'id': 'call_3c29', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1272, 'output_tokens': 93, 'total_tokens': 1365})]

In [11]:
from langchain_core.messages import HumanMessage, ToolMessage

for tool_call in ai_msg.tool_calls:
    selected_tool = {"python_repl": python_repl}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

model_with_tools.invoke(messages)

RESULT CODE EXECUTION: SyntaxError('invalid syntax', ('<string>', 1, 1, './testfile.py\n', 1, 2))


AIMessage(content='False', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 1407, 'total_tokens': 1409, 'completion_time': 0.003125, 'prompt_time': 0.148214539, 'queue_time': None, 'total_time': 0.151339539}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-1cc3a946-a4c1-422b-ab3e-2f909ba78ba6-0', usage_metadata={'input_tokens': 1407, 'output_tokens': 2, 'total_tokens': 1409})

Work With Agents

In [12]:
from typing import TypedDict


class AgentState(TypedDict):
    message: str
    error: bool
    error_message: str
    file_path: str
    code: str
    iterations: int